The task of leraning semantic audio embeddings feel daunting - maybe we can help our model by pretraining the encoder to learn to distinguish words from audio?

This is providing a crutch to our model - it no longer will be unsupervised in the sense that we will levarge word labels for the pretraining. It might nonetheless be very useful to do as we work towards a fully working end to end unsupervised model.

In [1]:
from fastai.vision.all import *
from fastai.learner import *
from fastai.data.all import *
from fastai.callback.tracker import SaveModelCallback
import pandas as pd
import matplotlib.pyplot as plt
from pathlib2 import Path
import numpy as np
import random
from torch.nn import MSELoss

In [2]:
%%time

df = pd.read_csv('data/examples_with_length_speech2vec_vocab.csv')
df.shape

CPU times: user 31.8 s, sys: 3.69 s, total: 35.5 s
Wall time: 54.3 s


(17937758, 12)

In [3]:
df = df[((df.source_length < 70) & (df.target_length < 70) & (df.source_length > 25) & (df.target_length > 25))]
df.reset_index(drop=True, inplace=True)

In [4]:
%%time
fn2features = pd.read_pickle('data/fn2feature.pkl')

CPU times: user 17.5 s, sys: 11.1 s, total: 28.6 s
Wall time: 28.6 s


In [5]:
dataset_mean = -3
dataset_std = 12

def normalize_data(ary):
    return (ary - dataset_mean) / dataset_std

In [6]:
def empty_list(): return list()

In [7]:
# %%time

# word2row_idxs = defaultdict(empty_list)

# for idx, row in df.iterrows():
#     word2row_idxs[row.source_word].append(idx)
    
# pd.to_pickle(word2row_idxs, 'data/word2row_idxs_vocab_subset.pkl')

In [8]:
word2row_idxs = pd.read_pickle('data/word2row_idxs_vocab_subset.pkl')

In [9]:
def prepare_features(fn, pad_to=69, pad_left=False):
    ary = fn2features[fn][:pad_to]
    example = np.zeros((pad_to, 13))
    if pad_left:
        example[-ary.shape[0]:, :] = ary
    else: example[:ary.shape[0], :] = ary
    return example.astype(np.float32)

In [10]:
vocab = df.target_word.unique().tolist()

In [11]:
class Dataset():
    def __init__(self, df):
        self.df = df
    def __len__(self):
        return self.df.shape[0]
    def __getitem__(self, idx):
        target_fn = self.df.target_fn[idx]
        x = normalize_data(prepare_features(target_fn))
        return x, vocab.index(self.df.target_word[idx])

In [12]:
train_examples = df[df.set_name.isin(['train-clean-360', 'train-clean-100', 'dev-clean'])]
valid_examples = df[df.set_name == 'test-clean']

train_examples.reset_index(inplace=True, drop=True)
valid_examples.reset_index(inplace=True, drop=True)

In [13]:
train_ds = Dataset(train_examples)
valid_ds = Dataset(valid_examples)

In [14]:
len(train_ds), len(valid_ds)

(3300062, 36832)

In [15]:
BS = 2048
LR = 1e-3
NUM_WORKERS = 8

train_dl = DataLoader(train_ds, BS, NUM_WORKERS, shuffle=True)
valid_dl = DataLoader(valid_ds, BS, NUM_WORKERS)

dls = DataLoaders(train_dl, valid_dl)

In [16]:
# bidirectional encoder, 1 layer, concatenate hidden state
class Model(Module):
    def __init__(self, hidden_size=25, num_layers_encoder=3):
        self.return_embeddings = False
        self.num_layers_encoder = num_layers_encoder
        self.hidden_size = hidden_size
        
        self.encoder= nn.LSTM(
            input_size=13,
            hidden_size=hidden_size,
            num_layers=self.num_layers_encoder,
            batch_first=True,
            dropout=0,
            bidirectional=True
        ) 
        
        self.lin = nn.Linear(2*hidden_size, len(vocab))
            
    def forward(self, source_features):
        _, (embeddings, _) = self.encoder(source_features)
        
        embeddings = torch.cat((embeddings[-1], embeddings[-2]), 1)
        if self.return_embeddings: return embeddings
        
        return self.lin(embeddings)

In [17]:
learn = Learner(dls.cuda(), Model().cuda(), loss_func=CrossEntropyLossFlat(), lr=1e-3, opt_func=Adam, metrics=[accuracy])

In [18]:
learn.fit(30, lr=1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,6.249473,6.229773,0.133308,02:39
1,4.242486,4.317686,0.318962,02:41
2,3.207394,3.340269,0.447899,02:43
3,2.581912,2.761657,0.527395,02:42
4,2.199839,2.421428,0.576998,02:44
5,1.925077,2.185860,0.615525,02:42
6,1.739560,2.020813,0.638548,02:43
7,1.581429,1.886971,0.660241,02:42
8,1.467972,1.790694,0.672513,02:44
9,1.378024,1.716751,0.683753,02:42


In [19]:
learn.save('encoder_weights_supervised')

Path('models/encoder_weights_supervised.pth')

In [20]:
preds, targs = learn.get_preds()

In [26]:
valid_examples.head()

,source_word,target_word,source_fn,target_fn,set_name,speaker_id,book_id,distance_from_target,audio_fpath,source_length,target_length,in_speech2vec_vocab
0,STEW,DINNER,5186b91b28ab4c8c9534119e9714ac43,de23b34fbbac4791b7be6d6b0b63f835,test-clean,1089,134686,2,data/LibriSpeech/test-clean/1089/134686/1089-134686-0000.flac,38,36,True
1,DINNER,STEW,de23b34fbbac4791b7be6d6b0b63f835,5186b91b28ab4c8c9534119e9714ac43,test-clean,1089,134686,2,data/LibriSpeech/test-clean/1089/134686/1089-134686-0000.flac,36,38,True
2,DINNER,TURNIPS,de23b34fbbac4791b7be6d6b0b63f835,ef10f0583db242e087c2642c1a787df5,test-clean,1089,134686,1,data/LibriSpeech/test-clean/1089/134686/1089-134686-0000.flac,36,48,True
3,TURNIPS,DINNER,ef10f0583db242e087c2642c1a787df5,de23b34fbbac4791b7be6d6b0b63f835,test-clean,1089,134686,1,data/LibriSpeech/test-clean/1089/134686/1089-134686-0000.flac,48,36,True
4,TURNIPS,CARROTS,ef10f0583db242e087c2642c1a787df5,b007359ef51944b29b5489e062d85fb6,test-clean,1089,134686,2,data/LibriSpeech/test-clean/1089/134686/1089-134686-0000.flac,48,52,True


In [29]:
from IPython.lib.display import Audio

In [33]:
idx = 100
preds[idx].argmax(), targs[idx], valid_examples.target_fn[idx]

(tensor(3867), tensor(3867), '68893b7edf834d97a3818e8c509a65a9')

In [34]:
vocab[targs[idx]]

'PAGE'

In [35]:
Audio(data=f'data/examples_audio/{valid_examples.target_fn[idx]}.wav')